In [2]:
import numpy as np
np.set_printoptions(linewidth = 200, precision = 4, suppress = True)

In [51]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(float)
raw_data = np.linalg.cholesky(data).T
data.shape

(101, 101)

In [4]:
np.allclose(raw_data.T @ raw_data, data)

True

In [5]:
# Gershgorin upper bound on data dominant eigenvalue.
def zd(mat):
    mat = mat.copy()
    np.fill_diagonal(mat, 0)
    return mat
max(np.abs(zd(data)).sum(axis=1))

37.86214335491857

In [6]:
max(np.linalg.eigvalsh(data))

25.582479205536025

In [7]:
nodes = np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
nodes = nodes[1:].astype(float)
bounds = np.loadtxt('../NC-K7-Trace-Bounds.csv', delimiter=',', dtype=str)
bounds = bounds[1:].astype(float)

In [8]:
# current = np.where((nodes == 1.).sum(axis=0) == 2)[0][0]
current_state = 40
selected = np.where(nodes[:, current_state] == 1.)[0]
current_state, bounds[:, current_state], selected

(40, array([4.6441, 6.6205, 7.    ]), array([83, 85]))

In [9]:
# TODO: Treat properly before decomposition first zeroing out these other entries.
np.sum(nodes[:, current_state] == 0)

19

In [10]:
np.allclose(raw_data[:, selected].T @ raw_data[:, selected], data[selected, :][:, selected])

True

In [11]:
rank_one_left_vector = np.linalg.svd(raw_data[:, selected])[0][:, 0:1]
np.linalg.svd(raw_data[:, selected])[1]

array([1.4102, 0.1068])

In [34]:
rank_one_updates = rank_one_left_vector.T @ raw_data[:, nodes[:, current_state] != 0]
rank_one_updates

array([[-0.0173,  0.1751, -0.2929,  0.1267,  0.5301,  0.0424, -0.3041, -0.2094, -0.2214, -0.2264,  0.0207,  0.3938,  0.5109, -0.1673, -0.3748, -0.0699,  0.627 ,  0.2888,  0.3845,  0.4127,  0.5586,
        -0.1473,  0.5738,  0.5413, -0.1236, -0.0265, -0.5765,  0.5895, -0.4053,  0.0228, -0.3617, -0.3963,  0.1032, -0.1819, -0.0858, -0.0948,  0.1427, -0.0795, -0.0712, -0.0431, -0.0511,  0.3653,
         0.3695,  0.3885,  0.3816, -0.2469,  0.1397, -0.0142,  0.0271,  0.2503,  0.287 ,  0.0239, -0.2841,  0.197 , -0.0882,  0.2573, -0.322 , -0.3081,  0.1181, -0.3215,  0.9014,  0.889 ,  0.8674,
         0.9599,  0.9971,  0.9823,  0.9971,  0.0674,  0.6304,  0.0342, -0.0386,  0.0078,  0.4814, -0.4117,  0.0932, -0.0928, -0.1377, -0.0514,  0.2106,  0.3542, -0.0331, -0.2409]])

In [47]:
rank_one_proj_data = rank_one_updates.T @ rank_one_updates
np.linalg.eigvalsh(rank_one_proj_data)

array([-0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    ,
       -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    , 12.8443])

In [14]:
residual_data = raw_data - rank_one_left_vector @ rank_one_left_vector.T @ raw_data
np.linalg.matrix_rank(residual_data)

100

In [15]:
residual_data.T @ residual_data

array([[ 0.9997, -0.0431,  0.2261, ...,  0.2765,  0.4658,  0.363 ],
       [-0.0431,  0.9693, -0.0158, ..., -0.1135, -0.0886,  0.0073],
       [ 0.2261, -0.0158,  0.9142, ...,  0.2508,  0.2511,  0.5607],
       ...,
       [ 0.2765, -0.1135,  0.2508, ...,  0.8745,  0.2184,  0.2391],
       [ 0.4658, -0.0886,  0.2511, ...,  0.2184,  0.9989,  0.3406],
       [ 0.363 ,  0.0073,  0.5607, ...,  0.2391,  0.3406,  0.942 ]])

In [16]:
np.linalg.svd(residual_data)[1]

array([4.4848, 3.0916, 2.8783, 2.4244, 2.0752, 1.9912, 1.76  , 1.5649, 1.3619, 1.2528, 1.223 , 1.1864, 1.0376, 0.98  , 0.9477, 0.9309, 0.8695, 0.8458, 0.8151, 0.7996, 0.7732, 0.7443, 0.7366, 0.716 ,
       0.6974, 0.6836, 0.6762, 0.6554, 0.6336, 0.6175, 0.6142, 0.5951, 0.5794, 0.5478, 0.538 , 0.5337, 0.5157, 0.5002, 0.4949, 0.481 , 0.4701, 0.4586, 0.4501, 0.4386, 0.4246, 0.4064, 0.401 , 0.3767,
       0.3757, 0.355 , 0.3379, 0.3297, 0.3253, 0.3171, 0.2994, 0.2866, 0.2782, 0.2714, 0.2636, 0.2582, 0.2529, 0.2489, 0.244 , 0.2271, 0.2242, 0.2184, 0.2132, 0.2044, 0.2007, 0.1951, 0.1888, 0.1835,
       0.1784, 0.1773, 0.1674, 0.1603, 0.1549, 0.15  , 0.1463, 0.1419, 0.136 , 0.1332, 0.1312, 0.1245, 0.1161, 0.1084, 0.0967, 0.0916, 0.0773, 0.0766, 0.0721, 0.0641, 0.0616, 0.0522, 0.0451, 0.0421,
       0.0361, 0.0333, 0.0284, 0.0251, 0.    ])

In [17]:
data_zd = np.abs(zd(data)[:, nodes[:, current_state] == -1])
data_zd = np.sort(data_zd, axis=1)
max(data_zd[:, -5:].sum(axis=1))

4.749309933185232

In [18]:
data_zd = np.abs(zd(residual_data.T @ residual_data)[:, nodes[:, current_state] == -1])
data_zd = np.sort(data_zd, axis=1)
max(data_zd[:, -5:].sum(axis=1))

4.537432268659807

In [19]:
np.linalg.svd(residual_data)[0][:, 0:1].T @ rank_one_left_vector

array([[0.]])

In [20]:
np.linalg.svd(residual_data)[2][0:1, :] @ np.linalg.svd(rank_one_left_vector @ rank_one_left_vector.T @ raw_data)[2][0:1, :].T

array([[0.2378]])

In [31]:
data_zd = np.abs(zd(data[nodes[:, current_state] != 0, :][:, nodes[:, current_state] != 0]))
data_zd = np.sort(data_zd, axis=1)
max(data_zd[:, -7:].sum(axis=1))

6.262943805388188

In [75]:
data_zd = np.abs(zd((residual_data.T @ residual_data)[nodes[:, current_state] != 0, :][:, nodes[:, current_state] != 0]))
data_zd = np.sort(data_zd, axis=1)
max(data_zd[:, -7:].sum(axis=1))

5.994257985010179

### Ipsen et. al's perturbation bound

Start with the rank one matrix, and apply a perturbation (residuals). We can slice K rows and columns of the rank-one matrix, and easily compute the new eigenvector to be perturbed. The benefit is that we can multiply the perturbation matrix on the right by a normed eigenvector. However, we don't actually know which elements will be chosen, and only want to take those K elements from the eigenvector and norm that vector. Gershgorin code was adapted, however, we actually need to norm K elements of the vector after matrix multplication, which we did not do here. So the correct implementation might similarly perform worse than plain Gershgorin.

In [49]:
(residual_data.T @ residual_data)[nodes[:, current_state] != 0, :][:, nodes[:, current_state] != 0]

array([[ 0.9997, -0.0431,  0.2261, ...,  0.2765,  0.4658,  0.363 ],
       [-0.0431,  0.9693, -0.0158, ..., -0.1135, -0.0886,  0.0073],
       [ 0.2261, -0.0158,  0.9142, ...,  0.2508,  0.2511,  0.5607],
       ...,
       [ 0.2765, -0.1135,  0.2508, ...,  0.8745,  0.2184,  0.2391],
       [ 0.4658, -0.0886,  0.2511, ...,  0.2184,  0.9989,  0.3406],
       [ 0.363 ,  0.0073,  0.5607, ...,  0.2391,  0.3406,  0.942 ]])

In [72]:
rank_one_updates_normalizer = 1. / np.linalg.norm(sorted(rank_one_updates[0, :])[0:7])
data_zd = rank_one_updates_normalizer * np.abs(rank_one_updates * (residual_data.T @ residual_data)[nodes[:, current_state] != 0, :][:, nodes[:, current_state] != 0])
data_zd += (rank_one_updates ** 2)
data_zd = zd(data_zd)
data_zd = np.sort(data_zd, axis=-1)
max(data_zd[:, -7:].sum(axis=1))

6.643969368018722

In [111]:
# Can we brute-force Ipsen et. al until we have an analytic solution to optimize subset?
import scipy.special
scipy.special.binom(101 - 2, 5)

71523144.0

In [115]:
ipsen_subset = np.zeros(101, bool)
ipsen_subset[np.asarray(list(selected) + [0, 1, 2, 3, 4])] = 1

In [136]:
# Exact solution (can't do this combinatorially)
np.linalg.eigvalsh(data[ipsen_subset, :][:, ipsen_subset]).max()

2.60530318106641

In [137]:
# Gershgorin solution (not a tight upper bound)
abs(data[ipsen_subset, :][:, ipsen_subset]).sum(axis=1).max()

3.144985719959683

In [138]:
# Ipsen: L2 norm of general update mat * unperturbed eigenvector.
# We are dealing with terms which are square rooted vs terms which are not, but
# maybe we can select optimal subset without brute-force.
rank_one_trace = (rank_one_updates ** 2)[0, ipsen_subset[nodes[:, current_state] != 0]].sum()
ipsen_matrix = (residual_data.T @ residual_data)[ipsen_subset, :][:, ipsen_subset]
ipsen_vec = rank_one_updates[:, ipsen_subset[nodes[:, current_state] != 0]].T
ipsen_vec /= np.linalg.norm(ipsen_vec)
rank_one_trace + np.linalg.norm(ipsen_matrix @ ipsen_vec)

2.750902237607125

### Zhu's rank-one update bounds

These bounds require the neighboring eigenvalue, and not only the eigenvalue of interest. Smaller neighboring eigenvalue must be upper-bounded, because a smaller gap leads to more involvement in the update. Upper bound on the neighbor (using the trace) requires lower bound on the dominant eigenvalue.

If the gap is 0, then the quality of the bound degrades to adding the trace of the rank-one matrix.

In [102]:
data_zd = np.abs(zd((residual_data.T @ residual_data)[nodes[:, current_state] != 0, :][:, nodes[:, current_state] != 0]))
data_zd = np.sort(data_zd, axis=1)
residual_data_gersh = max(data_zd[:, -7:].sum(axis=1))
data_trunc = np.linalg.svd(residual_data)[2][0:1, :].T
data_trunc[np.argsort(np.abs(data_trunc[:, 0]))[:-7]] = 0
# data_trunc = residual_data.T @ residual_data @ data_trunc
# data_trunc[np.argsort(np.abs(data_trunc[:, 0]))[:-7]] = 0
data_trunc[data_trunc[:, 0] != 0, 0] = np.linalg.svd(residual_data[:, data_trunc[:, 0] != 0])[2][0, :]
data_trunc /= np.linalg.norm(data_trunc)

resid_trunc = residual_data.copy()
resid_trunc[data_trunc[:, 0] == 0, :] = 0
resid_trunc[:, data_trunc[:, 0] == 0] = 0
resid_trunc_power = resid_trunc.T @ resid_trunc @ data_trunc
residual_data_power = (data_trunc.T @ resid_trunc_power)[0, 0]
zhu_gap = residual_data_power - (7. - residual_data_power)
residual_data_gersh, residual_data_power, zhu_gap

(5.994257985010179, 3.138000398147341, -0.7239992037053176)

In [106]:
# What if the gap is at least 1?
# Also need to guess x_1 upper bound (cosine similarity of residual )
# Guess 0.5 for now.
residual_data_gersh + (np.linalg.norm(rank_one_updates) - 1. + np.sqrt(
    (1. + np.linalg.norm(rank_one_updates)) ** 2
    - 4 * 1. * (1. - 0.5 ** 2)
)) / 2.

9.408235196250924